In [ ]:
#%pip install ipyannotations

In [ ]:
#from ipyannotations.images import BoxAnnotator
#box_widget = BoxAnnotator(options=["decedent_name"])
#box_widget.display("Capture1.png")
#box_widget

In [ ]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#%pip install jupyter_bbox_widget

In [ ]:
from jupyter_bbox_widget import BBoxWidget
import os
import base64
import ipywidgets as widgets
cwd = os.getcwd()

state = 'Florida'
path = cwd + '/' + state + '/'
path

In [ ]:
from os import walk
mypath = cwd + '/' + state + '/'
filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file
filenames = [i for i in filenames if i.endswith('pdf')]

In [ ]:
import random
randomsample = random.sample(range(len(filenames)), 200)
pdfsample = [filenames[i] for i in randomsample]

In [ ]:
def ocr(state, doc):
    path = cwd + '/' + state + '/'
    pdf_pages = convert_from_path(path + doc,300)
    c=0
    image_file_list = []
    for page_enumeration, page in enumerate(pdf_pages, start=1):
        # enumerate() "counts" the pages for us.
        # Create a file name to store the image
        filename = r"{}.jpg".format(doc.split('.pdf')[0])
        print(filename)
        # Declaring filename for each page of PDF as JPG
        # Save the image of the page in system
        page.save(cwd + '/FloridaYoloImages/' + filename, "JPEG")
        image_file_list.append(cwd + '/FloridaYoloImages/' + filename)
        return image_file_list

In [ ]:
import os
import io
from PIL import Image as PILImage
import pytesseract
from wand.image import Image as wi
import gc
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

for pdf_fname in pdfsample: imlist = ocr(state, pdf_fname)
imlist

In [ ]:
#%pip install pdf2image
from PIL import Image
mypath = cwd + '/FloridaYoloImages/' 
filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file

size = 1200, 1554

for infile in filenames:
    outfile = mypath + infile
    im = Image.open(mypath + infile)
    im.thumbnail(size, Image.Resampling.LANCZOS)
    im.save(outfile, "JPEG")


In [ ]:
path = mypath
files = sorted(os.listdir(path))
print(files)
files = [i for i in files if i.lower()[-3:] in ['png', 'jpg']]
print(files)         
annotations = {}
annotations_path = 'annotations.json'

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    print('encode image works')
    return "data:image/jpg;base64,"+encoded
     
    
w_progress = widgets.IntProgress(value=0, max=len(files), description='Progress')
# the bbox widget
w_bbox = BBoxWidget(
    image= encode_image(path + files[0]),
    classes=['class1',],
)
@w_bbox.on_skip
def skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = encode_image(os.path.join(path, image_file))
    # here we assign an empty list to bboxes but 
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = [] 

# when Submit button is pressed we save current annotations
# and then move on to the next file
@w_bbox.on_submit
def submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    skip()

In [ ]:
# annotate bounding boxes here
#from jupyter_bbox_widget import BBoxWidget
w_bbox

In [ ]:
print(len(annotations), annotations)


In [ ]:

import json
"""
with open('annotationsNov25.json', 'w') as fp:
    json.dump(annotations, fp)
"""

In [ ]:
#annotations = json.load('annotations1.json')

with open('annotationsNov25.json') as f:
    annotations = json.load(f)
    print(annotations)

In [ ]:
w_bbox.bboxes

In [ ]:
import ipywidgets as widgets

int_slider = widgets.IntSlider(max=10, value=5)
int_slider

In [ ]:
#!git clone https://github.com/ultralytics/yolov5  # clone
#cd yolov5
%pip install -qr yolov5/requirements.txt comet_ml  # install

In [ ]:
import torch
#import utils
#display = utils.notebook_init()  # checks

In [ ]:
# 
import pandas as pd
import ast
!mkdir /mnt/batch/tasks/shared/LS_root/mounts/clusters/avanplatinga-gpu-nv6/code/Users/AVanPlantinga/OCR/yolov5/images
!mkdir /mnt/batch/tasks/shared/LS_root/mounts/clusters/avanplatinga-gpu-nv6/code/Users/AVanPlantinga/OCR/yolov5/labels

path = r'yolov5/labels/'

annot_dict = annotations
cc=0
for k in list(annotations.keys()):
    a = annotations[k][0]
    #print(a)
    im = k.split('.')[0]
    t = ''
    try: f = open(path + "{}.txt".format(im), "x")
    except: pass
    f.close()
    f = open(path + "{}.txt".format(im), "a")
    t += '0 {0} {1} {2} {3} \n'.format(a['x']/1200, a['y']/1554, a['width']/1200, a['height']/1554)
    #print(f)
    f.write('0 {0} {1} {2} {3}'.format(a['x']/1200, a['y']/1554, a['width']/1200, a['height']/1554))
    

In [ ]:
#!pip install tensorflow

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

In [ ]:
import os
fnamepath = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/avanplatinga-gpu-nv6/code/Users/AVanPlantinga/OCR/yolov5/images/'
f = open(fnamepath + "filenames.txt", "w")
f.close()
f = open(fnamepath + "filenames.txt", "a")
cc=0
for i in list(annotations.keys()):
    impath = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/avanplatinga-gpu-nv6/code/Users/AVanPlantinga/OCR/FloridaYoloImages/' + i
    os.system("cp {0} {1}".format(impath, fnamepath + i))

In [ ]:
yamlinput = """
            # YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
            # COCO128 dataset https://www.kaggle.com/ultralytics/coco128 (first 128 images from COCO train2017) by Ultralytics
            # Example usage: python train.py --data coco128.yaml
            # parent
            # ├── yolov5
            # └── datasets
            #     └── coco128  ← downloads here (7 MB)

            # Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
            path: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/avanplatinga-gpu-nv6/code/Users/AVanPlantinga/OCR/yolov5/'# dataset root dir
            train: images # train images (relative to 'path') 128 images
            val: images # val images (relative to 'path') 128 images
            test: # test images (optional)
            # Classes
            names:
              0: class1 #
            """

f = open("yolov5/newyolo_conf.yml", "w")
f.close()
f = open("yolov5/newyolo_conf.yml", "a")
f.write(yamlinput)
f.close()


In [ ]:
!pip install numpy==1.23.4
!pip uninstall comet-ml -y
#import numpy as np
#!pip install gitpython
#!pip uninstall protobuf -y
#!pip install protobuf 
#!tensorboard --logdir yolov5/runs/train

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
#!tensorboard --logdir yolov5/runs/train
#!git config --global --add safe.directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/avanplatinga-gpu-nv6/code/Users/AVanPlantinga/OCR/yolov5
!python yolov5/train.py --img 1568 --batch 8 --epochs 10 --data yolov5/newyolo_conf.yml --weights yolov5s.pt --cache

In [ ]:
#!ls

In [ ]:
!python yolov5/detect.py --weights yolov5/runs/train/exp5/weights/best.pt --img 1568 --conf 0.1 --source Florida